In [34]:
from bs4 import BeautifulSoup
from requests import get

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import math 
import os

In [35]:
# Function for remove comma within numbers
def removeCommas(string): 
    string = string.replace(',','')
    return string 

# Scrap data from worldmeter

In [36]:
# Test if we can scrap info from worldometers
# The communication with website is ok if the response is 200
headers = ({'User-Agent':'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36'})
worldometers = "https://www.worldometers.info/coronavirus/#countries"
response = get(worldometers, headers=headers)
response

<Response [200]>

In [37]:
# Scrap all content from the website
html_soup = BeautifulSoup(response.text, 'html.parser')
# After inspect the website content, data are stored inside tag 'tbody' and table header is 'thead'
table_contents = html_soup.find_all('tbody')
table_header = html_soup.find_all('thead')

# Header for the table
header = []
for head_title in table_header[0].find_all('th'):    
    header.append(str(head_title.contents))

# Save value into columns
CountryName = []
TotalCases = []
NewCases = []
TotalDeaths = []
NewDeaths = []
TotalRecovered = []
ActiveCases = []
SeriousCritical = []

for row in table_contents[0].find_all('tr'):
    cells = row.find_all('td')
    if len(cells[0].find_all('a')) >= 1:
        CountryName.append(cells[0].find_all('a')[0].contents[0])
    elif len(cells[0].find_all('span')) >= 1:
        CountryName.append(cells[0].find_all('span')[0].contents[0])   
    else:
        CountryName.append(cells[0].contents[0])
    
    
    if len(cells[1].contents) >=1:
        TotalCases.append(cells[1].contents[0])
    else:
        TotalCases.append(0)
    
    if len(cells[2].contents) >= 1:
        NewCases.append(cells[2].contents[0])
    else:
        NewCases.append(0)
        
    
    if len(cells[3].contents) >= 1:
        TotalDeaths.append(cells[3].contents[0])
    else:
        TotalDeaths.append(0)

    
    if len(cells[4].contents) >= 1:
        NewDeaths.append(cells[4].contents[0])
    else:
        NewDeaths.append(0)
    
    if len(cells[5].contents) >= 1:
        TotalRecovered.append(cells[5].contents[0])
    else:
        TotalRecovered.append(0)
        
    if len(cells[6].contents) >= 1:
        ActiveCases.append(cells[6].contents[0])
    else:
        ActiveCases.append(0)
    
    if len(cells[7].contents) >= 1:
        SeriousCritical.append(cells[7].contents[0])
    else:
        SeriousCritical.append(0)
        
        
CaseTable = pd.DataFrame({header[0]: CountryName,
                          header[1]: TotalCases,
                          header[2]: NewCases,
                          header[3]: TotalDeaths,
                          header[4]: NewDeaths,                          
                          header[5]: TotalRecovered,
                          header[6]: ActiveCases,
                          header[7]: SeriousCritical,
                          })  

CaseTable.head(40)

,"['Country,', <br>Other</br>]","['Total', <br>Cases</br>]","['New', <br>Cases</br>]","['Total', <br/>, 'Deaths']","['New', <br/>, 'Deaths']","['Total', <br/>, 'Recovered']","['Active', <br/>, 'Cases']","['Serious,', <br/>, 'Critical']"
0,China,"81,285",+67,"3,287",+6,"74,051","3,947","1,235"
1,Italy,"74,386",0,"7,503",0,"9,362","57,521","3,489"
2,USA,"68,594",+383,"1,036",+9,428,"67,130","1,455"
3,Spain,"56,188","+6,673","4,089",+442,"7,015","45,084","3,166"
4,Germany,"39,502","+2,179",222,+16,"3,547","35,733",23
5,Iran,"29,406","+2,389","2,234",+157,"10,457","16,715","2,746"
6,France,"25,233",0,"1,331",0,"3,900","20,002","2,827"
7,Switzerland,"11,435",+538,165,+12,131,"11,139",141
8,UK,"9,529",0,465,0,135,"8,929",163
9,S. Korea,"9,241",+104,131,+5,"4,144","4,966",59


In [38]:
CaseTable.tail(40)

,"['Country,', <br>Other</br>]","['Total', <br>Cases</br>]","['New', <br>Cases</br>]","['Total', <br/>, 'Deaths']","['New', <br/>, 'Deaths']","['Total', <br/>, 'Recovered']","['Active', <br/>, 'Cases']","['Serious,', <br/>, 'Critical']"
159,Guyana,5,0,1,0,0,4,0
160,Bahamas,5,0,,0,1,4,0
161,Fiji,5,0,,0,0,5,0
162,Mozambique,5,0,,0,0,5,0
163,Syria,5,0,,0,0,5,0
164,Cabo Verde,4,0,1,0,0,3,0
165,Congo,4,0,,0,0,4,0
166,Eritrea,4,0,,0,0,4,0
167,Guinea,4,0,,0,0,4,0
168,Vatican City,4,0,,0,0,4,0


In [39]:
caseTableSimple = CaseTable[[CaseTable.columns[0], CaseTable.columns[1], CaseTable.columns[3], CaseTable.columns[5]]]
caseTableSimple.columns = ['Country/Region', 'Confirmed', 'Deaths', 'Recovered']
# Set data type as string first for manuipulation
caseTableSimple = caseTableSimple.astype({'Country/Region':str,'Confirmed':str,'Deaths':str, 'Recovered':str})
# Remove the last row of total number (changed on 20200310, worldmeter moved this row as next tbody)
#caseTableSimple = caseTableSimple.iloc[:-1,:]
# Remove lead and tail space for each element
caseTableSimple = caseTableSimple.apply(lambda x: x.str.strip())
# Remove comma for each element
caseTableSimple = caseTableSimple.applymap(removeCommas)
# Replace empty str with zero. This include row of 'Diamond Princess' (its name is empty)
caseTableSimple = caseTableSimple.replace('', '0')
# After string manipulation, convert data type as correct type
caseTableSimple = caseTableSimple.astype({'Country/Region':'str',
                                          'Confirmed':'int',
                                          'Deaths':'int',
                                          'Recovered':'int',                                          
                                         })
# Data for these countries come from other source
removeRegion = ['China', 'Canada', 'Australia', 'USA']
for i in removeRegion:
    caseTableSimple.drop(caseTableSimple[caseTableSimple['Country/Region'] == i].index, axis=0, inplace=True)

# Change Country name the same as my old data 
if 'S. Korea' in list(caseTableSimple['Country/Region']):
    caseTableSimple = caseTableSimple.replace('S. Korea', 'South Korea')

# Add column 'Province/State' with empty value
caseTableSimple['Province/State'] =''

# In my old data, 'Diamond Princess' is represented by 'Yokohama' in the column of 'Province/State'
if 'Diamond Princess' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Diamond Princess',].index, 'Province/State'] = 'Yokohama'
    caseTableSimple['Country/Region'].replace({'Diamond Princess':'Japan'}, inplace=True)

# In my old data, 'Belgium' has 'Brussels' in the column of 'Province/State'
if 'Belgium' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Belgium',].index, 'Province/State'] = 'Brussels'

# In my old data, I used 'Macau' not 'Macao'
if 'Macao' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Macao',].index, 'Province/State'] = 'Macau'
    caseTableSimple['Country/Region'].replace({'Macao':'Macau'}, inplace=True)

# In my old data, 'Hong Kong' has 'Hong Kong' in the column of 'Province/State'
if 'Hong Kong' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Hong Kong',].index, 'Province/State'] = 'Hong Kong'

# In my old data, 'Taiwan' has 'Taiwan' in the column of 'Province/State'
if 'Taiwan' in list(caseTableSimple['Country/Region']):
    caseTableSimple.at[caseTableSimple.loc[caseTableSimple['Country/Region'] == 'Taiwan',].index, 'Province/State'] = 'Taiwan'

# In my old data, I used 'United Arab Emirates' not 'UAE'
if 'UAE' in list(caseTableSimple['Country/Region']):
    caseTableSimple['Country/Region'].replace({'UAE':'United Arab Emirates'}, inplace=True)

if 'Réunion' in list(caseTableSimple['Country/Region']):
    caseTableSimple['Country/Region'].replace({'Réunion':'Reunion'}, inplace=True)
    
if 'Curaçao' in list(caseTableSimple['Country/Region']):
    caseTableSimple['Country/Region'].replace({'Curaçao':'Curacao'}, inplace=True)

# In my old data I used US time as Last Update time
currentTime = datetime.now()
lastUpdateTime = currentTime.strftime('%m/%d/%Y %H:%M')
# Remove the first number (This only works for month number less than 10)
lastUpdateTime[1:]
caseTableSimple['Last Update'] = lastUpdateTime[1:]

# Reorder list as all old data
columnList = caseTableSimple.columns.tolist()
columnList =[columnList[i] for i in [4, 0, 5, 1, 2, 3]]
caseTableSimple = caseTableSimple[columnList]

In [40]:
caseTableSimple.tail(20)

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
179,,Mauritania,3/26/2020 22:54,3,0,0
180,,Myanmar,3/26/2020 22:54,3,0,0
181,,St. Barth,3/26/2020 22:54,3,0,0
182,,Saint Lucia,3/26/2020 22:54,3,0,0
183,,Sint Maarten,3/26/2020 22:54,3,0,0
184,,Belize,3/26/2020 22:54,2,0,0
185,,Bhutan,3/26/2020 22:54,2,0,0
186,,British Virgin Islands,3/26/2020 22:54,2,0,0
187,,Guinea-Bissau,3/26/2020 22:54,2,0,0
188,,Mali,3/26/2020 22:54,2,0,0


# Scrap data for US_CAN

In [41]:
# Test if we can scrap info from worldometers
# The communication with website is ok if the response is 200
US_Canada = "https://coronavirus.1point3acres.com/zh"
response2 = get(US_Canada, headers=headers)
response2

<Response [200]>

In [42]:
# Scrap all content from the website
html_soup2 = BeautifulSoup(response2.text, 'html.parser')

In [43]:
# Since they change class index everyday, this code is for finding the new index.
indexList = []
for span in html_soup2.find_all('span'):
    # Only retain 'span' that has contents
    if len(span.contents):
        # Since we only need to find index for table, use one of the table head as target word to locate index
        if span.contents[0] == 'Location':
            # Store the index inside a list
            indexList.append(span['class'][0])

In [44]:
# The first index is for US table and the 2nd index is for Canada table. Do not care about the rest inside the list.
USindex, CANindex = indexList

In [45]:
# Check if the index return right data
html_soup2.find_all('span', class_=USindex)

[<span class="jsx-564222390" style="color:#ee7500">🎉 Click here to check the state testing data/location</span>,
 <span class="jsx-564222390">Location</span>,
 <span class="jsx-564222390">Confirmed</span>,
 <span class="jsx-564222390">Deaths</span>,
 <span class="jsx-564222390">Fatality rate</span>,
 <span class="jsx-564222390">Source</span>,
 <span class="jsx-564222390">United States</span>,
 <span class="jsx-564222390">69222</span>,
 <span class="jsx-564222390">1050</span>,
 <span class="jsx-564222390">1.5%</span>,
 <span class="jsx-564222390"><a class="jsx-564222390" href="https://www.cdc.gov" target="_blank">CDC</a></span>,
 <span class="jsx-564222390">New York<i aria-label="icon: home" class="anticon anticon-home" tabindex="-1"><svg aria-hidden="true" class="" data-icon="home" fill="currentColor" focusable="false" height="1em" viewbox="64 64 896 896" width="1em"><path d="M946.5 505L560.1 118.8l-25.9-25.9a31.5 31.5 0 0 0-44.4 0L77.5 505a63.9 63.9 0 0 0-18.8 46c.4 35.2 29.7 63.3 64.

In [46]:
Locations = []
Confirmed = []
Recovered = []
Deaths = []
list1 = range(1, len(html_soup2.find_all('span', class_=USindex))-4, 5)
list2 = range(2, len(html_soup2.find_all('span', class_=USindex))-3, 5)
list3 = range(3, len(html_soup2.find_all('span', class_=USindex))-2, 5)
list4 = range(4, len(html_soup2.find_all('span', class_=USindex))-1, 5)

for index in list1:
    if len(html_soup2.find_all('span', class_=USindex)[index].contents):
        Locations.append(html_soup2.find_all('span', class_=USindex)[index].contents[0])
    else:
        Locations.append(0)
for index in list2:
    if len(html_soup2.find_all('span', class_=USindex)[index].contents):
        try:
            Confirmed.append(html_soup2.find_all('span', class_=USindex)[index].contents[1])
        except:
            Confirmed.append(html_soup2.find_all('span', class_=USindex)[index].contents[0])
    else:
        Confirmed.append(0)
for index in list3:
    # They do not provide recovered cases number anymore.
    #if len(html_soup2.find_all('span', class_=USindex)[index].contents):
    #    Recovered.append(html_soup2.find_all('span', class_=USindex)[index].contents[0])
    #else:
    Recovered.append(0)
for index in list3:
    if len(html_soup2.find_all('span', class_=USindex)[index].contents):
        try:
            Deaths.append(html_soup2.find_all('span', class_=USindex)[index].contents[1])
        except:
            Deaths.append(html_soup2.find_all('span', class_=USindex)[index].contents[0])
    else:
        Deaths.append(0)
    
US_data = pd.DataFrame({'Province/State':Locations,
                        'Confirmed':Confirmed,
                        'Deaths':Deaths,
                        #'Recovered':Recovered,  
                            })

# Remove rows that are not data
US_data.drop(US_data[US_data['Deaths'] == 'Deaths'].index, axis=0, inplace=True)

# Remove rows that are not data
US_data.drop(US_data[US_data['Province/State'] == 'United States'].index, axis=0, inplace=True)

# Replace Washington, D.C. as Washington DC
if 'Washington, D.C.' in list(US_data['Province/State']):
    US_data['Province/State'].replace({'Washington, D.C.':'Washington DC'}, inplace=True)

# Replace Washington as WA
if 'Washington' in list(US_data['Province/State']):
    US_data['Province/State'].replace({'Washington':'WA'}, inplace=True)
    
# Replace Grand Princess as From Grand Princess
#if 'Grand Princess' in list(US_data['Province/State']):
#    US_data['Province/State'].replace({'Grand Princess':'From Grand Princess'}, inplace=True)
    
# Replace Diamond Princess as From Diamond Princess cruise
#if 'Diamond Princess' in list(US_data['Province/State']):
#    US_data['Province/State'].replace({'Diamond Princess':'From Diamond Princess cruise'}, inplace=True)
    
# Assign 0 in column Province/State as unassigned
if 0 in list(US_data['Province/State']):
    US_data.at[US_data.loc[US_data['Province/State'] == 0,].index, 'Province/State'] = 'Unassigned'

In [47]:
US_data

,Province/State,Confirmed,Deaths
2,New York,33033,366
3,New Jersey,4407,62
4,California,3169,67
5,WA,2600,133
6,Michigan,2297,43
7,Florida,1978,23
8,Illinois,1870,19
9,Massachusetts,1838,15
10,Louisiana,1795,65
11,Georgia,1387,47


In [48]:
# As the website changed to dynamic, using selenium to interact with the website vitually
from selenium import webdriver

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [49]:
# Open vitual Chrome browser
driver = webdriver.Chrome()
# Direct the driver to open a webpage by calling the ‘get’ method, with a parameter of the page we want to visit.
driver.get("https://coronavirus.1point3acres.com/en")
# click tab button to let page lode new data (US data is the default)
python_button = driver.find_element(By.XPATH, "//span[text()='Canada']")
python_button.click()
# Wait for the dynamically loaded elements to show up
WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.CLASS_NAME, CANindex)))
# And grab the new page HTML source
html_page = driver.page_source
driver.quit()

In [50]:
# Now we can use html_page as source for BS4
html_soup2 = BeautifulSoup(html_page)

In [51]:
Locations = []
Confirmed = []
Recovered = []
Deaths = []
list1 = range(0, len(html_soup2.find_all('span', class_=CANindex))-4, 5)
list2 = range(1, len(html_soup2.find_all('span', class_=CANindex))-3, 5)
list3 = range(2, len(html_soup2.find_all('span', class_=CANindex))-2, 5)
list4 = range(3, len(html_soup2.find_all('span', class_=CANindex))-1, 5)

for index in list1:
    if len(html_soup2.find_all('span', class_=CANindex)[index].contents):
        Locations.append(html_soup2.find_all('span', class_=CANindex)[index].contents[0])
    else:
        Locations.append(0)
for index in list2:
    if len(html_soup2.find_all('span', class_=CANindex)[index].contents):
        try:
            Confirmed.append(html_soup2.find_all('span', class_=CANindex)[index].contents[1])
        except:
            Confirmed.append(html_soup2.find_all('span', class_=CANindex)[index].contents[0])
    else:
        Confirmed.append(0)
for index in list3:
    #. They do not provide recovered cases number
    #if len(html_soup2.find_all('span', class_=CANindex)[index].contents):
    #    Recovered.append(html_soup2.find_all('span', class_=CANindex)[index].contents[0])
    #else:
    Recovered.append(0)
for index in list3:
    if len(html_soup2.find_all('span', class_=CANindex)[index].contents):
        try:
            Deaths.append(html_soup2.find_all('span', class_=CANindex)[index].contents[1])
        except:
            Deaths.append(html_soup2.find_all('span', class_=CANindex)[index].contents[0])
    else:
        Deaths.append(0)
    
CAN_data = pd.DataFrame({'Province/State':Locations,
                         'Confirmed':Confirmed,
                         'Deaths':Deaths,
                         #'Recovered':Recovered,  
                            })

# Remove rows that are not data
CAN_data.drop(CAN_data[CAN_data['Deaths'] == 'Deaths'].index, axis=0, inplace=True)

# Remove rows that are not data
CAN_data.drop(CAN_data[CAN_data['Province/State'] == 'Canada'].index, axis=0, inplace=True)

In [52]:
CAN_data

,Province/State,Confirmed,Deaths
2,Quebec,1342,6
3,Ontario,688,12
4,British Columbia,659,14
5,Alberta,419,2
6,Saskatchewan,86,0
7,Nova Scotia,68,0
8,Newfoundland and Labrador,67,0
9,Manitoba,35,0
10,New Brunswick,18,0
11,Grand Princess,13,0


In [53]:
US_Can_data = pd.concat([US_data, CAN_data], ignore_index=True)
US_Can_data = US_Can_data.apply(lambda x: x.str.strip())
US_Can_data

,Province/State,Confirmed,Deaths
0,New York,33033,366
1,New Jersey,4407,62
2,California,3169,67
3,WA,2600,133
4,Michigan,2297,43
...,...,...,...
66,Grand Princess,13,0
67,Prince Edward Island,5,0
68,Yukon,3,0
69,Northwest Territories,1,0


In [54]:
nameList = pd.read_csv('./web_data/statesNameTranslation.csv')

In [55]:
US_Can_data_EN = pd.merge(US_Can_data, nameList, how = 'left', 
                          left_on = 'Province/State', 
                          right_on = 'English')
US_Can_data_EN = US_Can_data_EN.drop(['Chinese', 'Province/State', 'Abbr.'], axis=1)
US_Can_data_EN['Last Update'] = lastUpdateTime[1:]
US_Can_data_EN.rename(columns={'English':'Province/State'}, inplace=True)
US_Can_data_EN = US_Can_data_EN.drop(US_Can_data_EN[US_Can_data_EN['Province/State'] == 'Wuhan Evacuee'].index, axis=0)
columnOrder = ['Province/State', 'Country/Region', 'Last Update','Confirmed', 'Deaths', 'Recovered']

US_Can_data_EN = US_Can_data_EN[columnOrder]
US_Can_data_EN 

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,New York,US,3/26/2020 22:54,33033,366,0
1,New Jersey,US,3/26/2020 22:54,4407,62,1
2,California,US,3/26/2020 22:54,3169,67,6
3,WA,US,3/26/2020 22:54,2600,133,1
4,Michigan,US,3/26/2020 22:54,2297,43,0
...,...,...,...,...,...,...
66,Grand Princess,US,3/26/2020 22:54,13,0,0
67,Prince Edward Island,Canada,3/26/2020 22:54,5,0,0
68,Yukon,Canada,3/26/2020 22:54,3,0,0
69,Northwest Territories,Canada,3/26/2020 22:54,1,0,0


In [56]:
finalTable = pd.concat([US_Can_data_EN, caseTableSimple], ignore_index=True)
finalTable

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,New York,US,3/26/2020 22:54,33033,366,0
1,New Jersey,US,3/26/2020 22:54,4407,62,1
2,California,US,3/26/2020 22:54,3169,67,6
3,WA,US,3/26/2020 22:54,2600,133,1
4,Michigan,US,3/26/2020 22:54,2297,43,0
...,...,...,...,...,...,...
260,,Montserrat,3/26/2020 22:54,1,0,0
261,,Papua New Guinea,3/26/2020 22:54,1,0,0
262,,St. Vincent Grenadines,3/26/2020 22:54,1,0,0
263,,Timor-Leste,3/26/2020 22:54,1,0,0


In [57]:
timeStampe = currentTime.strftime('%m_%d_%Y_%H_%M')
finalTable.to_csv('./web_data/{}_webData.csv'.format(timeStampe), index=False)

# Scrap data for China

In [58]:
# Test if we can scrap info from worldometers
# The communication with website is ok if the response is 200
#CHN = "https://ncov.dxy.cn/ncovh5/view/pneumonia?scene=2&clicktime=1579582238&enterid=1579582238&from=singlemessage&isappinstalled=0"
#response3 = get(CHN, headers=headers)
#response3.encoding='utf-8' ##去掉这句则乱码，加上则正常显示，其中utf-8是根据网页源代码中设置的编码格式来指定的  
#response3

In [59]:
# Scrap all content from the website
#html_soup3 = BeautifulSoup(response3.text, 'html.parser')

In [60]:
#print(html_soup3.prettify())

In [61]:
#html_soup3.find_all('script', id='getAreaStat')[0].contents